In [388]:
import pandas as pd
import numpy as np

In [389]:
### DEFINE BETA VALUES FOR VDF

beta_vals = {} # TBD: compute values based on growth

beta_template_aux = {
    "Period": ["AM", "MD", "PM", "NT"],
    1: [3.246, 3.249, 3.399, 3.402],
    2: [0.715, 0.708, 0.592, 0.708],
    3: [3.130, 3.133, 3.283, 3.286]
}

beta_vals_aux = pd.DataFrame(beta_template_aux)

beta_vals_aux.set_index("Period", inplace=True)

beta_vals[2025] = beta_vals_aux

In [390]:
### HERE ARE THE MEASURED SPEEDS WE USE AS REFERENCE

measured_speeds_file = r"inputs/measured_speeds.csv"

measured_speeds = pd.read_csv(
    measured_speeds_file,
    sep=",",          # `delimiter` y `sep` son equivalentes; elige uno
    encoding="utf-8",
    decimal=".",      # parsea decimales con punto
    thousands=",",    # parsea separador de miles con coma
    quotechar='"',
    index_col=0
)

measured_speeds

,1SB,1NB,2SB,2NB,3SB,3NB,4SB,4NB,5SB,5NB,...,11SB,11NB,12SB,12NB,13SB,13NB,14SB,14NB,15SB,15NB
Capacity Factors,,,,,,,,,,,,,,,,,,,,,
Night,58.082011,59.162467,65.189993,66.412059,60.868315,65.865456,58.412998,64.977297,62.782079,66.351164,...,62.083005,66.122436,63.679802,63.416580,58.720702,58.081203,53.379413,53.882120,59.878663,62.784750
AM-Early,60.354772,60.309657,66.842800,68.473113,64.249888,65.865456,59.370588,66.776801,65.770031,69.366243,...,66.591983,67.196500,64.776931,68.372249,53.941650,59.785387,53.757349,55.172948,61.079114,60.645929
AM-Peak,58.419009,42.934403,45.518717,67.331894,41.451541,41.764687,59.865343,57.387883,66.076922,52.857489,...,65.338115,20.486993,64.197411,39.366724,26.857624,56.864919,50.586184,42.234153,61.079114,17.975508
AM-Shoulder,53.942842,18.864769,12.820656,66.228092,13.143720,29.148006,57.485807,29.284522,62.701652,28.381039,...,64.927183,12.928947,63.439354,32.731396,17.876012,55.589921,47.450621,23.102722,59.382472,14.852064
MD,46.180720,27.721968,34.638967,63.257198,20.120112,36.592436,55.240268,57.642540,58.230384,51.567091,...,60.476010,45.616500,64.046938,63.888842,29.346851,44.977262,32.804647,32.175390,55.116160,26.103247
PM-Shoulder,40.497737,25.543313,40.063628,50.408905,21.482935,36.462330,39.611314,62.189992,43.430750,54.265106,...,36.838118,47.728355,63.815849,62.426836,24.696177,34.820061,25.278754,20.305782,54.350059,34.291939
PM-Peak,19.583891,18.748770,41.115977,25.069010,21.034189,20.117277,11.894600,55.443556,23.365609,63.391106,...,26.267105,47.642989,52.323778,60.353679,19.187202,18.646202,21.614965,13.522268,54.504903,43.421925
PM-Late,37.708370,39.992438,54.539665,52.348735,44.239389,49.247384,31.838411,61.347934,34.796289,64.055767,...,57.247588,61.677172,63.817348,64.354485,41.487969,37.762854,37.272148,47.118859,56.000776,53.728869


In [391]:
### DEFINE LOOKUP TABLE FOR BONUS PER PERIOD

lookup_period_file = r"inputs/LookUp_Period.csv"

lookup_period = pd.read_csv(
    lookup_period_file,
    sep=",",          # `delimiter` y `sep` son equivalentes; elige uno
    encoding="utf-8",
    decimal=".",      # parsea decimales con punto
    thousands=",",    # parsea separador de miles con coma
    quotechar='"',
    index_col=0
)

# Clip y reasignar
lookup_period = lookup_period*0

lookup_period

,Bonus/Mile,4 Periods
Period,,
Night,0.0,
AM-Early,0.0,
AM-Peak,0.0,
AM-Shoulder,0.0,
MD,0.0,
PM-Shoulder,0.0,
PM-Peak,0.0,
PM-Late,0.0,


In [392]:
### DEFINE SEGMENT PARAMETERS
# Default configuration for time periods in traffic data

#TBD: Make this automatically
period_template = [                 # (Period, Hours/Day, Peak/OP, 4Periods tag)
    ("Night",        6, "OP",   "NT"),
    ("AM-Early",     1, "OP",   "AM"),
    ("AM-Peak",      1, "Peak", "AM"),
    ("AM-Shoulder",  3, "OP",   "AM"),
    ("MD",           4, "OP",   "MD"),
    ("PM-Shoulder",  2, "OP",   "PM"),
    ("PM-Peak",      2, "Peak", "PM"),
    ("PM-Late",      5, "OP",   "PM"),
]

rows = []
years = [2025]

# Default time periods list (for reference)
default_time_periods = [
    "Night",
    "AM-Early",
    "AM-Peak",
    "AM-Shoulder",
    "MD",
    "PM-Shoulder",
    "PM-Peak",
    "PM-Late"
]

# Create the base scenario: hour -> time period mapping
hour_to_period = {
    0: "Night",
    1: "Night",
    2: "Night",
    3: "Night",
    4: "Night",
    5: "AM-Early",
    6: "AM-Peak",
    7: "AM-Peak",
    8: "AM-Shoulder",
    9: "AM-Shoulder",
    10: "MD",
    11: "MD",
    12: "MD",
    13: "MD",
    14: "PM-Shoulder",
    15: "PM-Shoulder",
    16: "PM-Peak",
    17: "PM-Peak",
    18: "PM-Late",
    19: "PM-Late",
    20: "PM-Late",
    21: "PM-Late",
    22: "PM-Late",
    23: "Night"
}

period_to_period = {
    'Evening': 'Night',
    'Evening': 'PM-Late',
    'EarlyAM': 'AM-Early',
    'AM': 'AM-Peak',
    'AM': 'AM-Shoulder',
    'Midday': 'MD',
    'Midday': 'PM-Shoulder',
    'PM': 'PM-Peak'
}

# Define the segments and their parameters

peak_factor = 1 # 1.05 # Peak factor for adjustment at peak hour traffic

hov_percentage = pd.DataFrame({
    'Year' : [2025],
    'HOV percentage' : [0]
})

hov_percentage.set_index('Year', inplace=True)

# Define segment parameters base
seg_params = pd.DataFrame({
    'SegDir':   ["1NB","1SB","2NB","2SB","3NB","3SB","4NB","4SB","5NB","5SB","6NB","6SB","7NB","7SB","8NB","8SB","9NB","9SB","10NB","10SB"],
    'Length':    [0.7,0.7,0.7,0.7,0.5,0.5,1.6,1.6,2,2,3.6,3.6,2.9,2.9,3.8,3.8,3.4,3.4,4.5,4.5],
    'Inscope':   [0.8]*20,
    'Lanes_GP':  [4]*20, #
    'Lanes_ML':  [2]*20, # Lanes_ML': [2,2,2,2,2,2,2,2,3,3,2,2,2,2], # Do test changing segment 5
    'CapPerLane_GP': [2000]*20,
    'CapPerLane_ML': [1800]*20,
    'Speed_GP':  [55]*6 + [65]*2 + [70]*12,
    'Speed_ML':  [70]*20,
    'Alpha_GP':  [1]*20,
    'Beta_GP':   [6]*20,
    'Alpha_ML':  [1]*20,
    'Beta_ML':   [6]*20,
    'Min_Toll_2016': [None]*20,
    'Max_Toll_2016': [None]*20,
    'LanesGP_AM_Peak': [5]*20,
    'LanesGP_PM_Peak': [5]*20,
})

seg_params.set_index('SegDir', inplace=True)

# Compute capacities as lanes * cap per lane
seg_params['Cap_GP'] = seg_params['Lanes_GP'] * seg_params['CapPerLane_GP']
seg_params['Cap_ML'] = seg_params['Lanes_ML'] * seg_params['CapPerLane_ML']

# Compute peak capacities as Alpha * base capacity
seg_params['CapGP_Peak'] = seg_params['Alpha_GP'] * seg_params['Cap_GP']
seg_params['CapML_Peak'] = seg_params['Alpha_ML'] * seg_params['Cap_ML']

# Optional: if you want integer capacities
seg_params[['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']] = seg_params[
    ['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']
].astype(int)

# Preview
seg_params

,Length,Inscope,Lanes_GP,Lanes_ML,CapPerLane_GP,CapPerLane_ML,Speed_GP,Speed_ML,Alpha_GP,Beta_GP,Alpha_ML,Beta_ML,Min_Toll_2016,Max_Toll_2016,LanesGP_AM_Peak,LanesGP_PM_Peak,Cap_GP,Cap_ML,CapGP_Peak,CapML_Peak
SegDir,,,,,,,,,,,,,,,,,,,,
1NB,0.7,0.8,4,2,2000,1800,55,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
1SB,0.7,0.8,4,2,2000,1800,55,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
2NB,0.7,0.8,4,2,2000,1800,55,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
2SB,0.7,0.8,4,2,2000,1800,55,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
3NB,0.5,0.8,4,2,2000,1800,55,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
3SB,0.5,0.8,4,2,2000,1800,55,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
4NB,1.6,0.8,4,2,2000,1800,65,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
4SB,1.6,0.8,4,2,2000,1800,65,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
5NB,2.0,0.8,4,2,2000,1800,70,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600


In [393]:
import numpy as np

def adjusted_cumprod(row, target_year, multiplier):
    years = row.index
    print(row.values)
    factors = 1 + row.values
    
    # Find the index of the target year
    target_idx = list(years).index(target_year)
    
    # Apply multiplier to the target year's factor
    factors[target_idx] *= multiplier
    
    # Calculate cumulative product
    return pd.Series(np.cumprod(factors), index=years)

In [394]:
### IMPORT GROWTHS FOR EACH CLASS
file_path_growths = r"inputs/growths_per_segment.csv"
base_growth_df = pd.read_csv(
    file_path_growths,
    delimiter=',',
    encoding='utf-8',
    decimal='.',        # ← this tells pandas how to parse decimals
    thousands=',',       # ← this tells pandas how to parse thousands
    quotechar='"'
)

base_growth_df = base_growth_df.iloc[:, 1:]
project_years = base_growth_df.columns[1:].tolist()
base_growth_df.iloc[:, 1:] =  base_growth_df.iloc[:, 1:] + 1

base_growth_df.loc[:, '2032'] *= 1.12

base_growth_df

,SegmentMapped,2026,2027,2028,2029,2030,2031,2032,2033,2034,...,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071
0,S1,1.018928,1.018928,1.018928,1.018928,1.018928,1.022899,1.145647,1.022899,1.022899,...,1.014078,1.014078,1.014078,1.014078,1.008447,1.008447,1.008447,1.008447,1.008447,1.008447
1,S2,1.025583,1.025583,1.025583,1.025583,1.025583,1.031514,1.155296,1.031514,1.031514,...,1.016660,1.016660,1.016660,1.016660,1.009996,1.009996,1.009996,1.009996,1.009996,1.009996
2,S3,1.023025,1.023025,1.023025,1.023025,1.023025,1.028939,1.152412,1.028939,1.028939,...,1.015544,1.015544,1.015544,1.015544,1.009326,1.009326,1.009326,1.009326,1.009326,1.009326
3,S4,1.022150,1.022150,1.022150,1.022150,1.022150,1.027914,1.151263,1.027914,1.027914,...,1.015180,1.015180,1.015180,1.015180,1.009108,1.009108,1.009108,1.009108,1.009108,1.009108
4,S5,1.018077,1.018077,1.018077,1.018077,1.018077,1.022963,1.145719,1.022963,1.022963,...,1.013537,1.013537,1.013537,1.013537,1.008122,1.008122,1.008122,1.008122,1.008122,1.008122
5,S6,1.018314,1.018314,1.018314,1.018314,1.018314,1.023435,1.146247,1.023435,1.023435,...,1.013597,1.013597,1.013597,1.013597,1.008158,1.008158,1.008158,1.008158,1.008158,1.008158
6,S7,1.020046,1.020046,1.020046,1.020046,1.020046,1.025430,1.148481,1.025430,1.025430,...,1.014297,1.014297,1.014297,1.014297,1.008578,1.008578,1.008578,1.008578,1.008578,1.008578
7,S8,1.019537,1.019537,1.019537,1.019537,1.019537,1.025049,1.148055,1.025049,1.025049,...,1.014060,1.014060,1.014060,1.014060,1.008436,1.008436,1.008436,1.008436,1.008436,1.008436
8,S9,1.017924,1.017924,1.017924,1.017924,1.017924,1.022940,1.145693,1.022940,1.022940,...,1.013443,1.013443,1.013443,1.013443,1.008066,1.008066,1.008066,1.008066,1.008066,1.008066
9,S10,1.020342,1.020342,1.020342,1.020342,1.020342,1.024674,1.147635,1.024674,1.024674,...,1.014523,1.014523,1.014523,1.014523,1.008714,1.008714,1.008714,1.008714,1.008714,1.008714


In [395]:
### IMPORT COUNTS AND SEPARATE BY CLASS AND PERIODS

file_path_counts = r"inputs/counts_by_hour_grouped_sorted.csv"
base_counts_df = pd.read_csv(
    file_path_counts,
    delimiter=',',
    encoding='utf-8',
    decimal='.',        # ← this tells pandas how to parse decimals
    thousands=',',       # ← this tells pandas how to parse thousands
    quotechar='"'
)

# --- Ajustar direcciones ---
base_counts_df["Direction"] = base_counts_df["Direction"].replace({"EB": "EB", "WB": "WB"})

# --- Crear columna Seg/Dir ---
base_counts_df["Seg/Dir"] = base_counts_df["Segment"].astype(str) + base_counts_df["Direction"]

# --- Función para procesar cada clase ---
def process_class(df_class):
    # Convertir a formato largo
    df_long = df_class.melt(
        id_vars=["Seg/Dir", "Segment", "Direction", "Class"],
        value_vars=[str(h) for h in range(24)],
        var_name="Hour",
        value_name="Volume"
    )
    
    # Mapear hora a periodo
    df_long["Hour"] = df_long["Hour"].astype(int)
    df_long["Period"] = df_long["Hour"].map(hour_to_period)
    
    # Agregar por Segment/Direction/Class/Period
    df_period = df_long.groupby(
        ["Seg/Dir", "Segment", "Direction", "Class", "Period"], as_index=False, sort=False
    ).agg({"Volume": "mean"}).round(0)
    
    # Pivot a formato ancho (periodos como columnas)
    period_order = df_period['Period'].unique()
    df_wide = df_period.pivot(
        index=["Seg/Dir", "Segment", "Direction", "Class"],
        columns="Period",
        values="Volume"
    )[period_order].reset_index()
    
    # Mantener solo Seg/Dir como índice
    df_proc = df_wide.drop(columns=["Class", "Direction", "Segment"]).set_index("Seg/Dir")
    
    return df_proc

# --- Separar por clases y procesar ---
dfs_by_class = {}
for cls in base_counts_df["Class"].unique():
    df_cls = base_counts_df[base_counts_df["Class"] == cls].copy()
    dfs_by_class[cls] = process_class(df_cls)


'''
Vehicle Classifications follow FHWA standards:
Lights: FHWA Classes 1-3 [Light Duty Vehicles]
Medium A: Classes 4-5 [Buses and Single Unit 2 axles trucks] 
Medium B: Class 6-7 [Single Unit 3 or 4 axles Trucks]
Heavy A: Classes 8-10 [Single Trailer 3 or more axles trucks]
Heavy B: Classes 11-13 [Combination Trucks Multitrailer Trucks]
'''

# --- Ejemplo de uso ---
df_lights = dfs_by_class["Lights"]
df_mediumA = dfs_by_class["Medium A"]
df_mediumB = dfs_by_class["Medium B"]
df_heavyA = dfs_by_class["Heavy A"]
df_heavyB = dfs_by_class["Heavy B"]

df_lights

Period,Night,AM-Early,AM-Peak,AM-Shoulder,MD,PM-Shoulder,PM-Peak,PM-Late
Seg/Dir,,,,,,,,
S10NB,692.0,4117.0,5323.0,3622.0,3423.0,4074.0,4314.0,2411.0
S10SB,613.0,1413.0,3661.0,3181.0,3528.0,5372.0,6426.0,2893.0
S1NB,982.0,4676.0,5314.0,3694.0,3666.0,3832.0,3686.0,3364.0
S1SB,1306.0,2435.0,3592.0,3132.0,4074.0,4112.0,3588.0,4084.0
S2NB,886.0,3926.0,5576.0,4446.0,4746.0,4996.0,4726.0,3617.0
S2SB,974.0,2492.0,3758.0,3971.0,5299.0,6208.0,6392.0,4067.0
S3NB,637.0,3032.0,3024.0,2598.0,3070.0,3367.0,3028.0,2334.0
S3SB,1060.0,2991.0,4321.0,4110.0,4694.0,4413.0,3506.0,3693.0
S4NB,786.0,5139.0,5852.0,4893.0,4544.0,5002.0,4945.0,2663.0


In [396]:
# --- Lista de periodos según tus columnas ---
period_cols = ["Night","AM-Early","AM-Peak","AM-Shoulder","MD","PM-Shoulder","PM-Peak","PM-Late"]

# Diccionario de dataframes por clase
class_dfs = {
    "Lights": df_lights,
    "Medium A": df_mediumA,
    "Medium B": df_mediumB,
    "Heavy A": df_heavyA,
    "Heavy B": df_heavyB
}

projected_long_by_class = {}

for cls_name, df_class in class_dfs.items():
    df = df_class.copy()
    
    # Resetear índice Seg/Dir y extraer Segment y Direction
    df = df.reset_index()
    df["Segment"] = df["Seg/Dir"].str.extract(r"(\d+)")[0]    # solo los números
    df["Direction"] = df["Seg/Dir"].str.extract(r"([A-Z]+)")[0]  # solo las letras
    df["Class"] = cls_name
    
    # Melt usando las columnas de periodos
    df_long = df.melt(
        id_vars=["Seg/Dir","Segment","Direction","Class"],
        value_vars=period_cols,
        var_name="Period",
        value_name="AADT "+str(df["Class"][0])
    )
    
    # Normalizar SegDir (opcional)
    df_long["SegDir"] = df_long["Seg/Dir"].str.strip().str.upper().str.lstrip("S")
    
    projected_long_by_class[cls_name] = df_long

# Ejemplo: ver Lights
projected_long_lights_df = projected_long_by_class["Lights"]
projected_long_mediumA_df = projected_long_by_class["Medium A"]
projected_long_mediumB_df = projected_long_by_class["Medium B"]
projected_long_heaviesA_df = projected_long_by_class["Heavy A"]
projected_long_heaviesB_df = projected_long_by_class["Heavy B"]
projected_long_lights_df


,Seg/Dir,Segment,Direction,Class,Period,AADT Lights,SegDir
0,S10NB,10,S,Lights,Night,692.0,10NB
1,S10SB,10,S,Lights,Night,613.0,10SB
2,S1NB,1,S,Lights,Night,982.0,1NB
3,S1SB,1,S,Lights,Night,1306.0,1SB
4,S2NB,2,S,Lights,Night,886.0,2NB
...,...,...,...,...,...,...,...
155,S7SB,7,S,Lights,PM-Late,3878.0,7SB
156,S8NB,8,S,Lights,PM-Late,2590.0,8NB
157,S8SB,8,S,Lights,PM-Late,3024.0,8SB
158,S9NB,9,S,Lights,PM-Late,2364.0,9NB


In [397]:
rows = []

for year in years:
    for seg in seg_params.index:  # e.g., "1NB", "1SB", etc.
        seg_data = seg_params.loc[seg]
        # Extraer parte numérica y dirección
        seg_numeric = ''.join(filter(str.isdigit, seg))  # e.g., "10"
        direction = seg[len(seg_numeric):]       
        for p, hrs, peak, tag in period_template:
            rows.append({
                "Year": year,
                "SegDir": seg,
                "Segment": seg_numeric,        
                "Direction": direction,     
                "Period": p,
                "Hours/Day": hrs,
                "Peak": peak,
                "4Periods": tag,

                # Parámetros técnicos
                "Length": seg_data["Length"],
                "Speed GP": seg_data["Speed_GP"],
                "Capacity GP": seg_data["CapPerLane_GP"] * seg_data["Lanes_GP"],
                "Alpha GP": seg_data["Alpha_GP"],
                "Beta GP": seg_data["Beta_GP"],
                "Speed ML": seg_data["Speed_ML"],
                "Capacity ML": seg_data["CapPerLane_ML"] * seg_data["Lanes_ML"],
                "Alpha ML": seg_data["Alpha_ML"],
                "Beta ML": seg_data["Beta_ML"],
                "MinToll": 0.5,
                "MinCapture": 0
            })

# --- plantilla base ---
first_model_df = pd.DataFrame(rows)

# --- merge para todas las clases ---
for cls_name, df_proj in projected_long_by_class.items():
    proj_merge_df = df_proj[["SegDir", "Period", f"AADT {cls_name}"]].copy()
    proj_merge_df.rename(columns={"AADT": f"AADT {cls_name}"}, inplace=True)
    
    first_model_df = first_model_df.merge(
        proj_merge_df,
        on=["SegDir", "Period"],
        how="left"
    )

# --- 1. Reshape growths a formato largo ---
growths_long = base_growth_df.melt(
    id_vars="SegmentMapped",
    var_name="Year",
    value_name="AnnualGrowth"
).copy()
growths_long["Year"] = growths_long["Year"].astype(int)

# --- 2. Calcular crecimiento acumulado desde 2025 ---
# Ordenamos por año y aplicamos cumprod
growths_long = growths_long.sort_values(["SegmentMapped", "Year"])
growths_long["GrowthFactor"] = (growths_long["AnnualGrowth"]).groupby(growths_long["SegmentMapped"]).cumprod()

# Ahora GrowthFactor(y) = factor acumulado 2025→y

# --- 3. Preparar plantilla ---
fm = first_model_df.copy()
fm["Year"] = fm["Year"].astype(int)
fm["Segment"] = fm["Segment"].astype(str).str.replace(r"^S", "", regex=True)
fm["SegmentMapped"] = "S" + fm["Segment"].astype(str)

# --- 4. Merge GrowthFactor ---
fm = fm.merge(
    growths_long[["SegmentMapped", "Year", "GrowthFactor"]],
    on=["SegmentMapped", "Year"],
    how="left"
)

fm["GrowthFactor"] = fm["GrowthFactor"].fillna(1.0)

# --- 5. Aplicar GrowthFactor a todas las clases ---
for cls_name in projected_long_by_class.keys():
    col = f"AADT {cls_name}"
    if col in fm.columns:
        fm[col] = (fm[col].fillna(0) * fm["GrowthFactor"]).round(1)

# --- 6. Limpieza ---
fm = fm.drop(columns=["SegmentMapped"])   # opcional

first_model_df = fm

first_model_df


,Year,SegDir,Segment,Direction,Period,Hours/Day,Peak,4Periods,Length,Speed GP,...,Alpha ML,Beta ML,MinToll,MinCapture,AADT Lights,AADT Medium A,AADT Medium B,AADT Heavy A,AADT Heavy B,GrowthFactor
0,2025,1NB,1,NB,Night,6,OP,NT,0.7,55,...,1,6,0.5,0,982.0,34.0,5.0,254.0,33.0,1.0
1,2025,1NB,1,NB,AM-Early,1,OP,AM,0.7,55,...,1,6,0.5,0,4676.0,70.0,46.0,292.0,20.0,1.0
2,2025,1NB,1,NB,AM-Peak,1,Peak,AM,0.7,55,...,1,6,0.5,0,5314.0,145.0,65.0,247.0,8.0,1.0
3,2025,1NB,1,NB,AM-Shoulder,3,OP,AM,0.7,55,...,1,6,0.5,0,3694.0,174.0,38.0,368.0,8.0,1.0
4,2025,1NB,1,NB,MD,4,OP,MD,0.7,55,...,1,6,0.5,0,3666.0,145.0,41.0,420.0,12.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2025,10SB,10,SB,AM-Shoulder,3,OP,AM,4.5,70,...,1,6,0.5,0,3181.0,204.0,58.0,470.0,22.0,1.0
156,2025,10SB,10,SB,MD,4,OP,MD,4.5,70,...,1,6,0.5,0,3528.0,152.0,57.0,496.0,16.0,1.0
157,2025,10SB,10,SB,PM-Shoulder,2,OP,PM,4.5,70,...,1,6,0.5,0,5372.0,169.0,56.0,412.0,12.0,1.0
158,2025,10SB,10,SB,PM-Peak,2,Peak,PM,4.5,70,...,1,6,0.5,0,6426.0,156.0,34.0,360.0,8.0,1.0


In [398]:
first_model_df["Capacity GP"] = first_model_df.apply(
    lambda row: seg_params.loc[row["SegDir"], 'Cap_GP'],
    axis=1
)

first_model_df["B1"] = first_model_df.apply(
    lambda row: beta_vals[row["Year"]].loc[row["4Periods"], 1],
    axis=1
)

first_model_df["B2"] = first_model_df.apply(
    lambda row: beta_vals[row["Year"]].loc[row["4Periods"], 2],
    axis=1
)

# Here we load th value of the counts and we multiply the peak hour values by a constant
lights_w = 1

heavies_w = 3
heavies_w_toll = 3
heavies_w_vot = 3

medium_A_w = 3 # TBD: Maybe try 2.5 or 2.75 for every pce value
medium_A_w_toll = 3
medium_A_w_vot = 3

medium_B_w = 3
medium_B_w_toll = 3
medium_B_w_vot = 3

heavy_A_w = 3
heavy_A_w_toll = 5
heavy_A_w_vot = 3

heavy_B_w = 3
heavy_B_w_toll = 3
heavy_B_w_vot = 3


# 2. Compute TotalLights
first_model_df["TotalLights"] = first_model_df["AADT Lights"] 

first_model_df["TotalMediumA"] = first_model_df["AADT Medium A"]

first_model_df["TotalMediumB"] = first_model_df["AADT Medium B"]

first_model_df["TotalHeavyA"] = first_model_df["AADT Heavy A"]

first_model_df["TotalHeavyB"] = first_model_df["AADT Heavy B"]

first_model_df["TotalVeh"] = first_model_df.apply(
    lambda row: row["TotalLights"] + row["TotalMediumA"] + row["TotalMediumB"] + row["TotalHeavyA"] + row["TotalHeavyB"],
    axis=1
)

first_model_df["Corridor PCE pre-fix"] = first_model_df.apply(
    lambda row: row["TotalLights"] * lights_w + row["TotalMediumA"] * medium_A_w + row["TotalMediumB"] * medium_B_w + row["TotalHeavyA"] * heavy_A_w + row["TotalHeavyB"] * heavy_B_w,
    axis=1
)

first_model_df["Corridor PCE"] = first_model_df["Corridor PCE pre-fix"] # To anulate the suppression


first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalLights'] *= peak_factor

first_model_df["HOV3"] = first_model_df.apply(
    lambda row: row["TotalLights"] * hov_percentage.loc[row['Year']]['HOV percentage'],
    axis=1
)

first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalMediumA'] *= peak_factor
first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalMediumB'] *= peak_factor
first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalHeavyA'] *= peak_factor
first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalHeavyB'] *= peak_factor

first_model_df["TotalVeh"] = first_model_df.apply(
    lambda row: row["TotalLights"] + row["TotalMediumA"] + row["TotalMediumB"] + row["TotalHeavyA"] + row["TotalHeavyB"],
    axis=1
)

first_model_df["InScopeLights"] = first_model_df.apply(
    lambda row: (row["TotalLights"] - row["HOV3"]) * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeMediumA"] = first_model_df.apply(
    lambda row: row["TotalMediumA"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeMediumB"] = first_model_df.apply(
    lambda row: row["TotalMediumB"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeHeavyA"] = first_model_df.apply(
    lambda row: row["TotalHeavyA"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeHeavyB"] = first_model_df.apply(
    lambda row: row["TotalHeavyB"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeVeh"] = first_model_df.apply(
    lambda row: row["TotalVeh"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

# first_model_df.to_csv('model_test.csv')

In [399]:
def get_speed(row):

    max_VC = 1.2  # TBD: check if we need to change this value
    ETC_discount = 0.15


    gp_pce = (
        row["TotalLights"] * lights_w + 
        row["TotalMediumA"] * medium_A_w +
        row["TotalMediumB"] * medium_B_w +
        row["TotalHeavyA"] * heavy_A_w +
        row["TotalHeavyB"] * heavy_B_w
    )

    speedGP =  row["Speed GP"] / (1 + row["Alpha GP"] * ((gp_pce / row["Capacity GP"]) ** row["Beta GP"]))

    timeGP = 60 * row["Length"] / speedGP

    gp_vc = gp_pce / row["Capacity GP"]

    return pd.Series([speedGP, timeGP], index=["Speed GP","Time GP"])

In [400]:
first_model_df[["Speed GP Real", "Time GP"]] = first_model_df.apply(
    get_speed, axis=1, result_type='expand'
)

first_model_df.to_csv('model_test.csv')

In [401]:
def get_pce(row):

    measured_speed = measured_speeds.loc[row['Period']][row['SegDir']]

    gp_pce = (
        row["TotalLights"] * lights_w + 
        row["TotalMediumA"] * medium_A_w +
        row["TotalMediumB"] * medium_B_w +
        row["TotalHeavyA"] * heavy_A_w +
        row["TotalHeavyB"] * heavy_B_w
    )

    speed_rate = row['Speed GP'] / measured_speed

    if speed_rate < 1:

        measured_speed = row['Speed GP'] - 0.01

    gp_pce_aux = row["Capacity GP"] * ((row['Speed GP'] / (measured_speed * row['Alpha GP']) - 1 / row['Alpha GP']) ** (1 / row['Beta GP']))

    if row["Period"] == "PM-Peak" or row["Period"] == "AM-Peak" or row["Period"] == "AM-Shoulder" or row["Period"] == "PM-Shoulder":

        pce_factor = gp_pce_aux / gp_pce

        pce_factor = np.clip(pce_factor, a_min = 1, a_max = 1.05)
    
    else:
        pce_factor = 1

    # pce_factor = gp_pce_aux / gp_pce

    # pce_factor = np.clip(pce_factor, a_min = 0.7, a_max = 1.3)

    return pce_factor

    

In [402]:
first_model_df["PCE Factor"] = first_model_df.apply(
    lambda row: get_pce(row),
    axis=1
)

# first_model_df.to_csv('model_test.csv')

In [403]:
cap_factor = first_model_df.pivot(
    index=["Period"],
    columns="SegDir",
    values="PCE Factor"
)

cap_factor.to_csv('inputs/pce_factors.csv')

cap_factor

SegDir,10NB,10SB,1NB,1SB,2NB,2SB,3NB,3SB,4NB,4SB,5NB,5SB,6NB,6SB,7NB,7SB,8NB,8SB,9NB,9SB
Period,,,,,,,,,,,,,,,,,,,,
AM-Early,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.000000,1.00,1.000000,1.00,1.00,1.00,1.000000,1.00,1.00000,1.000000,1.0
AM-Peak,1.0,1.0,1.00,1.00,1.00,1.05,1.05,1.05,1.00,1.030585,1.00,1.000000,1.00,1.00,1.05,1.000000,1.05,1.00000,1.006519,1.0
AM-Shoulder,1.0,1.0,1.05,1.00,1.00,1.05,1.05,1.05,1.05,1.050000,1.05,1.000000,1.05,1.00,1.05,1.000000,1.05,1.00000,1.050000,1.0
MD,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.000000,1.00,1.000000,1.00,1.00,1.00,1.000000,1.00,1.00000,1.000000,1.0
Night,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.000000,1.00,1.000000,1.00,1.00,1.00,1.000000,1.00,1.00000,1.000000,1.0
PM-Late,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.000000,1.00,1.000000,1.00,1.00,1.00,1.000000,1.00,1.00000,1.000000,1.0
PM-Peak,1.0,1.0,1.05,1.05,1.05,1.00,1.05,1.05,1.00,1.050000,1.00,1.050000,1.00,1.05,1.00,1.012714,1.00,1.01192,1.000000,1.0
PM-Shoulder,1.0,1.0,1.05,1.05,1.00,1.00,1.05,1.05,1.00,1.050000,1.00,1.041528,1.00,1.00,1.00,1.000000,1.00,1.00000,1.000000,1.0


In [ ]:
# period_map = {0: "Night", 1:"Night", ...}
period_map_series = pd.Series(hour_to_period)

dropped_columns = ['Segment', 'Direction', 'Class', 'AADT']

df_counts = base_counts_df.drop(columns=dropped_columns).groupby(['Seg/Dir']).sum()

df_counts.index = df_counts.index.str.lstrip('S')
       # your traffic counts dataframe
df_factors = cap_factor        # your period × segdir dataframe

hours = list(range(24))
output = df_counts.copy()

df_counts

for h in hours:
    period = period_map_series[h]           # "AM-Early", "MD", etc.
    factors = df_factors.loc[period]        # row of segdir factors

    output[str(h)] = df_counts.apply(
        lambda row: row[str(h)] * factors[row.name],
        axis=1
    )
df_counts.to_csv('base_total_counts.csv')
output.to_csv('new_counts_only_peaks.csv')

In [405]:
first_model_df["TotalLights"] = first_model_df.apply(
    lambda row: row["TotalLights"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

first_model_df["TotalMediumA"] = first_model_df.apply(
    lambda row: row["TotalMediumA"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

first_model_df["TotalMediumB"] = first_model_df.apply(
    lambda row: row["TotalMediumB"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

first_model_df["TotalHeavyA"] = first_model_df.apply(
    lambda row: row["TotalHeavyA"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

first_model_df["TotalHeavyB"] = first_model_df.apply(
    lambda row: row["TotalHeavyB"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

In [406]:
def get_capacity(row):

    measured_speed = measured_speeds.loc[row['Period']][row['SegDir']]

    gp_pce = (
        row["TotalLights"] * lights_w + 
        row["TotalMediumA"] * medium_A_w +
        row["TotalMediumB"] * medium_B_w +
        row["TotalHeavyA"] * heavy_A_w +
        row["TotalHeavyB"] * heavy_B_w
    )

    speed_rate = row['Speed GP'] / measured_speed

    if speed_rate < 1:

        measured_speed = row['Speed GP'] - 0.01

    capacity = gp_pce / ((row['Speed GP'] / (measured_speed * row['Alpha GP']) - 1 / row['Alpha GP']) ** (1 / row['Beta GP']))

    capacity_factor = capacity / row["Capacity GP"]

    return capacity_factor

In [407]:
first_model_df["Capacity Factor"] = first_model_df.apply(
    lambda row: get_capacity(row),
    axis=1
)

cap_factor = first_model_df.pivot(
    index=["Period"],
    columns="SegDir",
    values="Capacity Factor"
)

cap_factor.to_csv('inputs/capacity_factors.csv')

cap_factor

SegDir,10NB,10SB,1NB,1SB,2NB,2SB,3NB,3SB,4NB,4SB,5NB,5SB,6NB,6SB,7NB,7SB,8NB,8SB,9NB,9SB
Period,,,,,,,,,,,,,,,,,,,,
AM-Early,1.497384,0.609435,3.129946,2.077528,2.403652,1.700989,2.036566,2.256082,3.421367,0.648253,1.863419,0.708562,1.350432,0.679676,1.065057,0.740713,1.307776,1.016192,1.276502,0.579850
AM-Peak,1.735420,1.452239,1.036201,2.699316,3.333183,0.804798,0.614208,0.922429,1.245386,1.000000,1.130132,1.124935,1.006628,1.101973,0.812938,1.089795,0.791640,1.115314,1.000000,1.072221
AM-Shoulder,1.255744,1.276316,0.642815,1.185869,2.936688,0.555222,0.529848,0.657038,0.878436,0.980146,0.915770,1.105718,0.849411,1.040202,0.697135,1.000242,0.654869,1.014255,0.990179,1.020540
MD,1.175206,1.357152,0.691859,1.022421,3.103688,0.931441,0.630466,0.754653,1.184162,1.055790,1.070111,1.104663,1.051838,1.223413,1.010464,1.181496,1.012039,1.119611,1.002280,1.083059
Night,0.297618,0.301842,1.029311,1.207340,0.696886,0.744675,0.690584,0.952113,0.777153,0.328923,0.351486,0.328761,0.283302,0.283746,0.276125,0.303336,0.303328,0.312198,0.270351,0.265475
PM-Late,0.672457,0.866163,0.665998,0.760267,0.856127,1.293699,0.557271,0.734735,0.722950,0.585140,0.718020,0.643625,0.625137,0.694170,0.575927,0.741299,0.594810,0.658304,0.598686,0.702280
PM-Peak,1.161088,1.657187,0.574197,0.563987,0.701242,1.082856,0.470360,0.537896,1.040210,0.442167,1.237718,0.708025,1.140185,0.915901,1.141383,1.000000,1.049727,1.000000,1.025042,1.091545
PM-Shoulder,1.203399,1.956716,0.678012,0.870339,1.126704,1.074273,0.653769,0.692477,1.383203,0.803292,1.104096,1.000000,1.170751,1.229201,1.168975,1.309586,1.082801,1.195861,1.011501,1.359642


In [408]:
period_order = [
    "Night",
    "AM-Early",
    "AM-Peak",
    "AM-Shoulder",
    "MD",
    "PM-Shoulder",
    "PM-Peak",
    "PM-Late"
]

first_model_df["Total Corridor"] = first_model_df.apply(
    lambda row: row["Corridor PCE"] * row["Hours/Day"],
    axis=1
)


first_model_df["Period"] = pd.Categorical(
    first_model_df["Period"],
    categories=period_order,
    ordered=True
)

corridor_pce = first_model_df.pivot(
    index=["Period"],
    columns="SegDir",
    values="Total Corridor"
)

# corridor_pce.to_csv('corridor_vals.csv')